<a href="https://colab.research.google.com/github/rlcampos/Aulas/blob/master/Copy_of_Copy_of_Transfer_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<table style="width:100%" align="right">
  <tr>
    <td><center style="font-size:300%;">Transfer Learning</center></td>
    <td><img src="https://logodownload.org/wp-content/uploads/2015/02/puc-rio-logo.gif" width="100"/></td> 
  </tr>    
</table>

Msc. Cristian Muñoz V.

Para baixar o dataset do repositorio:

In [0]:
%%bash 

rm -rf yalefaces yalefaces.zip transfer_learning_utils.py
wget -O yalefaces.zip https://github.com/crismunoz/DeepLearningExamples/raw/master/week1/data/yalefaces.zip
wget -O transfer_learning_utils.py https://github.com/crismunoz/DeepLearningExamples/raw/master/week1/transfer_learning_utils.py
unzip -qq yalefaces.zip

--2019-04-27 13:30:25--  https://github.com/crismunoz/DeepLearningExamples/raw/master/week1/data/yalefaces.zip
Resolving github.com (github.com)... 192.30.253.112, 192.30.253.113
Connecting to github.com (github.com)|192.30.253.112|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/crismunoz/DeepLearningExamples/master/week1/data/yalefaces.zip [following]
--2019-04-27 13:30:30--  https://raw.githubusercontent.com/crismunoz/DeepLearningExamples/master/week1/data/yalefaces.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6203596 (5.9M) [application/zip]
Saving to: ‘yalefaces.zip’

     0K .......... .......... .......... .......... ..........  0% 1.02M 6s
    50K .......... .......... .......... .....

A estrutura de pastas do dataset é a seguiente:

-- nomedataset

    -- *classe1*

      -- img1.jpg

      -- img2.jpg
      ...
   
    -- *classe2*

       -- img1.jpg

       -- img2.jpg
       ...
       
    ... *cla..*


In [0]:
from keras.models import Model,Input
from keras.layers.core import Flatten, Dense, Dropout, Activation
from keras.layers.convolutional import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.applications import  VGG16
from keras.applications.vgg16 import preprocess_input,decode_predictions
from keras.optimizers import SGD
from keras.preprocessing.image import array_to_img, img_to_array, load_img
from keras.applications.vgg16 import preprocess_input
from keras import losses
from keras.optimizers import RMSprop

import pickle
import cv2
import numpy as np
from PIL import Image
from IPython.display import Image
from random import randint
import matplotlib.pylab as plt
from transfer_learning_utils import *

import datetime
now = datetime.datetime.now


Using TensorFlow backend.


# Abrir Modelo Pretreinado

*Modelo*:

In [0]:
pretrained_model = VGG16(weights="imagenet", include_top=True)

for i, layer in enumerate(pretrained_model.layers):
    print(i , layer.name, layer.output_shape)

Instructions for updating:
Colocations handled automatically by placer.
553467904/553467096 [==============================] - 6s 0us/step
0 input_1 (None, 224, 224, 3)
1 block1_conv1 (None, 224, 224, 64)
2 block1_conv2 (None, 224, 224, 64)
3 block1_pool (None, 112, 112, 64)
4 block2_conv1 (None, 112, 112, 128)
5 block2_conv2 (None, 112, 112, 128)
6 block2_pool (None, 56, 56, 128)
7 block3_conv1 (None, 56, 56, 256)
8 block3_conv2 (None, 56, 56, 256)
9 block3_conv3 (None, 56, 56, 256)
10 block3_pool (None, 28, 28, 256)
11 block4_conv1 (None, 28, 28, 512)
12 block4_conv2 (None, 28, 28, 512)
13 block4_conv3 (None, 28, 28, 512)
14 block4_pool (None, 14, 14, 512)
15 block5_conv1 (None, 14, 14, 512)
16 block5_conv2 (None, 14, 14, 512)
17 block5_conv3 (None, 14, 14, 512)
18 block5_pool (None, 7, 7, 512)
19 flatten (None, 25088)
20 fc1 (None, 4096)
21 fc2 (None, 4096)
22 predictions (None, 1000)


Escolher a camada adecuada para realizar transfer Learning


*Modelo Bottleneck*:

In [0]:
bottleneck_model=Model(inputs  = pretrained_model.input,
                       outputs = pretrained_model.get_layer('flatten').output)
bottleneck_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

# Dataset yalefaces

<center><img src="https://github.com/crismunoz/DeepLearningExamples/raw/master/week1/dataset/img/Alldataset.png" width="600"/></center>

Abrir o dataset :

In [0]:
(X_train,y_train) , (X_test,y_test) = load_data('yalefaces', 0.2)
print('train shape:{}'.format(X_train.shape))
print('test shape:{}'.format(X_test.shape))

Found 166 images belonging to 15 classes.
train shape:(132, 224, 224, 3)
test shape:(34, 224, 224, 3)


Realizamos o processo de data augmentation para os dados de treinamento e teste

In [0]:
train_datagen = ImageDataGenerator(rescale=1,
                               shear_range=0.2, 
                               zoom_range=0.2, 
                               horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1)

batch_size = 32
train_generator = train_datagen.flow(X_train, y_train, 
                                     batch_size=batch_size, 
                                     shuffle=False)

test_generator = test_datagen.flow(X_test, y_test, 
                                     batch_size=batch_size, 
                                     shuffle=False)

Geração de vetores de imagens (features)

In [0]:
train_steps_per_epoch = X_train.shape[0]//batch_size + 1
test_steps_per_epoch = X_test.shape[0]//batch_size + 1

increase_by=1
steps_per_epoch = train_steps_per_epoch*increase_by
train_features=[]
train_labels=[]
for i in range(steps_per_epoch):
    print("\rTraining Batches: {}/{}".format(i+1,steps_per_epoch), end="")
    x,y = next(train_generator)
    x=preprocess_input(x)
    train_features.append(bottleneck_model.predict(x))
    train_labels.append(y)

test_features=[]
test_labels=[]
for i in range(test_steps_per_epoch):
    print("\rTest Batches: {}/{}".format(i+1,test_steps_per_epoch), end="")
    x,y = next(test_generator)
    x=preprocess_input(x)
    test_features.append(bottleneck_model.predict(x))
    test_labels.append(y)

train_features = np.concatenate(train_features,axis=0)
train_labels = np.concatenate(train_labels,axis=0)
test_features = np.concatenate(test_features,axis=0)
test_labels = np.concatenate(test_labels,axis=0)

print('\nCreation of vectors image finished!')

Test Batches: 2/2
Creation of vectors image finished!


Como a extração de features é um processo demorado, se recomenda salvar  os features casso seja necessario restar o kernel.

In [0]:
train_data = (train_features,train_labels)
test_data = (test_features,test_labels)
pickle.dump(train_data, open('train_features.p','wb'))
pickle.dump(test_data, open('test_features.p','wb'))

In [0]:
(train_features,train_labels) = pickle.load( open( "train_features.p", "rb" ) )
(test_features,test_labels) = pickle.load( open( "test_features.p", "rb" ))

Definimos nosso classificador:

In [0]:
inputs = Input(bottleneck_model.get_layer('flatten').output_shape[1:])
x = Dense(128)(inputs)
x = Activation('relu')(x)
x = Dropout(0.5)(x)
x = Dense(15)(x)
x = Activation('softmax')(x)
model_cls = Model(inputs=inputs, outputs=x)
model_cls.summary()

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 25088)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               3211392   
_________________________________________________________________
activation_1 (Activation)    (None, 128)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 15)                1935      
_________________________________________________________________
activation_2 (Activation)    (None, 15)                0         
Total params: 3,213,327
Tra

Treinamos o modelo

In [0]:
batch_size=32
epochs=30
from keras import optimizers

optimizer = optimizers.Adam(lr=0.001)

model_cls.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

t = now()
history=model_cls.fit(train_features, train_labels,
                      batch_size=batch_size,
                      epochs=epochs,
                      verbose=1,
                      validation_data=(test_features, test_labels))

print('Training time: %s' % (now() - t))
scorelt5 = model_cls.evaluate(test_features, test_labels, verbose=0)
print('Test score:', scorelt5[0])
print('Test accuracy:', scorelt5[1])

Instructions for updating:
Use tf.cast instead.
Train on 132 samples, validate on 34 samples
Epoch 1/30
132/132 [==============================] - 1s 8ms/step - loss: 13.7365 - acc: 0.0985 - val_loss: 13.9057 - val_acc: 0.1176
Epoch 2/30
132/132 [==============================] - 0s 275us/step - loss: 12.1737 - acc: 0.1970 - val_loss: 13.1627 - val_acc: 0.1176
Epoch 3/30
132/132 [==============================] - 0s 285us/step - loss: 12.1491 - acc: 0.2121 - val_loss: 12.4919 - val_acc: 0.1765
Epoch 4/30
132/132 [==============================] - 0s 281us/step - loss: 11.8783 - acc: 0.2273 - val_loss: 11.3033 - val_acc: 0.2059
Epoch 5/30
132/132 [==============================] - 0s 273us/step - loss: 10.5027 - acc: 0.3030 - val_loss: 8.3088 - val_acc: 0.4118
Epoch 6/30
132/132 [==============================] - 0s 274us/step - loss: 10.2113 - acc: 0.3561 - val_loss: 8.7117 - val_acc: 0.4118
Epoch 7/30
132/132 [==============================] - 0s 274us/step - loss: 10.3109 - acc: 0.31

# Improving the model's performance:

Now try to improve the result you can change the hyperparameters, we encourage you to change the value of:

- epochs
- batch_size 
- increase_by
- learning rate